# <div align="center"><b> Desafio 4 - Procesamiento del lenguaje Natural - CEIA </b></div>


<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomndado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al sigiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/github/brunomaso1/uba-ceia/blob/ceia-nlp/ceia-nlp/Desafio%204.ipynb">Desafio 4</a></font></small></em> </div>


---


In [53]:
# Descargamos la carpeta con archivos auxiliares (Colab)
# %pip install gdown
# !gdown https://drive.google.com/drive/folders/1hNPqr6g3opu9u-UwnngVcyxUUzfmKZdT?usp=sharing --folder

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>


<div align="center"><img src="./resources/Desafio_4_portada.jpeg" width="600" alt="Figura 1: A data scientist is sitting in front of a computer screen, which is the focal point of the image. The lighting is dark and moody, with a blue hue. The computer screen displays a visualization of a QA bot. The bot and the scientist are chatting with each other, with the bot's responses appearing on the screen. - Generada con Microsoft Image generator"></div>

<div align="center"><small><em>Figura 1: A data scientist is sitting in front of a computer screen, which is the focal point of the image. The lighting is dark and moody, with a blue hue. The computer screen displays a visualization of a QA bot. The bot and the scientist are chatting with each other, with the bot's responses appearing on the screen. - Generada con Microsoft Image generator</em></small></div>


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Desafío 4 - NLP - FIUBA               |
| --------------- | ------------------------------------- |
| **Descrpción**  | BOT QA                                |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com) |

</div>


✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋


In [54]:
import os
import urllib.request
import json
import re
from IPython.display import display, HTML
import pandas as pd
import random
from tensorflow.keras.preprocessing.text import Tokenizer

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [55]:
RANDOM_SEED = 42
DATA_URL = "http://convai.io/data/data_volunteers.json"
LOCAL_FILENAME = "./resources/data_volunteers.json"
LABEL_SOS = "<sos> "
LABEL_EOS = " <eos>"
FILTERS = '!"#$%&()*+,-./:;=¿?@[\\]^_`{|}~\t\n'

random.seed(RANDOM_SEED)

## Consigna del desafío


El objetivo es utilizar datos disponibles del challenge [ConvAI2](http://convai.io/data/) (Conversational Intelligence Challenge 2) de conversaciones en inglés, para convertirlo en un BOT QA para responder preguntas del usuario.

### Parte 1

- Descargar el conjunto y pre-procesarlo.

### Parte 2 

- Realizar el preprocesamiento necesario para obtener:
    - `word2idx_inputs`, `max_input_len`
    - `word2idx_outputs`, `max_out_len`, `num_words_output`
    - `encoder_input_sequences`, `decoder_output_sequences`, `decoder_targets`

### Parte 3

- Utilizar los embeddings de *Glove* o *FastText* para transformar los tokens de entrada en vectores.

### Parte 4

- Entrenar un modelo basado en el esquema *encoder-decoder* utilizando los datos generados en los puntos anteriores.

### Parte 5

- Experimentar el modelo en inferencia.

## Resolución


### Parte 1

> - Descargar el conjunto y pre-procesarlo.

*Cargamos el conjunto:*

In [56]:
try:
    # Verificar si el archivo ya está descargado
    if not os.path.isfile(LOCAL_FILENAME):
        print(f"Archivo no encontrado localmente. Descargando desde {DATA_URL}...")
        urllib.request.urlretrieve(DATA_URL, LOCAL_FILENAME)
        print("Descarga completada.")
    else:
        print("El archivo ya existe localmente.")

    # Cargar el archivo JSON en la variable 'data'
    with open(LOCAL_FILENAME, "r", encoding="utf-8") as f:
        data = json.load(f)
    print("Archivo cargado correctamente.")
except urllib.error.URLError as e:
    print(f"Error al descargar el archivo: {e}")
except FileNotFoundError as e:
    print(f"Error al abrir el archivo: {e}")
except json.JSONDecodeError as e:
    print(f"Error al decodificar el archivo JSON: {e}")
except Exception as e:
    print(f"Se produjo un error inesperado: {e}")

El archivo ya existe localmente.
Archivo cargado correctamente.


*Mostramos los datos descargados:*

🔮 <em><font color='violet'>Función auxiliar:</font></em>
<em><font color='violet'><p>Función que limpia los textos..</p></font></em>

In [57]:
def clean_text(txt, contractions):
    # Convertimos todo el texto a minúsculas
    txt = txt.lower()
    
    # Reemplazamos contracciones comunes    
    for contraction, full_form in contractions.items():
        txt = txt.replace(contraction, full_form)
    
    # Eliminamos cualquier carácter no alfanumérico (y sustituimos por un espacio)
    txt = re.sub(r'\W+', ' ', txt)
    
    return txt.strip()

*Definimos el contexto máximo:*

In [58]:
MAX_LEN = 10

*Definimos las abrebiaciones identificadas:*

In [59]:
CONTRACTIONS = {"'d": " had", "'s": " is", "'m": " am", "don't": "do not"}

*Pre-procesamos el texto en sentencias:*

In [60]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

for line in data:
    dialog = line["dialog"]
    # Recorremos los diálogos pero limitamos a los pares de (input, respuesta)
    for i in range(len(dialog) - 1):
        # Limpiamos el texto de la "pregunta" y la "respuesta"
        chat_in = clean_text(dialog[i]["text"], CONTRACTIONS)
        chat_out = clean_text(dialog[i + 1]["text"], CONTRACTIONS)

        # Filtramos si alguna oración supera la longitud máxima permitida
        if len(chat_in.split()) >= MAX_LEN or len(chat_out.split()) >= MAX_LEN:
            continue

        # Agregamos las etiquetas <eos> y <sos> a las frases de salida
        output_sentence = chat_out + LABEL_EOS
        output_sentence_input = LABEL_SOS + chat_out

        # Añadimos las oraciones limpias a sus respectivas listas
        input_sentences.append(chat_in)
        output_sentences.append(output_sentence)
        output_sentences_inputs.append(output_sentence_input)

# Mostramos la cantidad de oraciones procesadas
print("Cantidad de rows utilizadas:", len(input_sentences))

Cantidad de rows utilizadas: 9092


*Chequeamos algunas sentencias:*

In [61]:
# Seleccionamos índices aleatorios
random_indices = random.sample(range(len(input_sentences)), 10)

# Creamos una lista para almacenar los datos
df_data = []

# Recorremos los índices aleatorios y creamos un diccionario para cada uno
for idx in random_indices:
    df_row = {
        "Indice": idx,
        "Input Sentence": input_sentences[idx],
        "Output Sentence": output_sentences[idx],
        "Output Sentence Input": output_sentences_inputs[idx]
    }
    df_data.append(df_row)

# Creamos el DataFrame con los datos
df = pd.DataFrame(df_data)

# Mostramos el DataFrame
display(HTML(df.to_html()))

,Indice,Input Sentence,Output Sentence,Output Sentence Input
0,1824,jonny depp,i know right i know right <eos>,<sos> i know right i know right
1,409,i am a pa how about you,i am a doctor and looking for new opportunities <eos>,<sos> i am a doctor and looking for new opportunities
2,4506,yes i have a lot of friends,how do they taste <eos>,<sos> how do they taste
3,4012,hyy,begin <eos>,<sos> begin
4,3657,what do you do for a living,job <eos>,<sos> job
5,2286,so you are divorced,yes i am i am a little overweight <eos>,<sos> yes i am i am a little overweight
6,1679,i like to read,why <eos>,<sos> why
7,8935,i am a student what do you do,i am aslo a student <eos>,<sos> i am aslo a student
8,1424,why,i love to read <eos>,<sos> i love to read
9,6912,yes,what do you do for a living <eos>,<sos> what do you do for a living


### Parte 2

> - Realizar el preprocesamiento necesario para obtener:  
    - `word2idx_inputs`, `max_input_len`  
    - `word2idx_outputs`, `max_out_len`, `num_words_output`  
    - `encoder_input_sequences`, `decoder_output_sequences`, `decoder_targets`  

*Primeramente, definimos el vocabulario máximo:*

In [62]:
MAX_VOCAB_SIZE = 8000

*Luego, tokenizamos las palabras de entrada:*

In [63]:
input_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print("Palabras en el vocabulario:", len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Sentencia de entrada más larga:", max_input_len)

Palabras en el vocabulario: 2724
Sentencia de entrada más larga: 9


*Y tokenizamos las palabras de salidas (target):*

In [67]:
output_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters=FILTERS)
output_tokenizer.fit_on_texts([str(LABEL_SOS.strip()), str(LABEL_EOS.strip)] + output_sentences)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print("Palabras en el vocabulario:", len(word2idx_outputs))

# Se suma 1 para incluir el token de palabra desconocida
num_words_output = min(len(word2idx_outputs) + 1, MAX_VOCAB_SIZE) 

max_out_len = max(len(sen) for sen in output_integer_seq)
print("Sentencia de salida más larga:", max_out_len)

Palabras en el vocabulario: 2737
Sentencia de salida más larga: 10
